## Imports

In [1]:
import pandas as pd
from my_model_package.forecasting.utils import pre_processing
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

## Load Data

In [2]:
df = pd.read_csv('blight_violations.csv')

/var/folders/5q/f280wjd15s56_y9s5ntbx3rw0000gn/T/ipykernel_77191/1277315867.py:1: DtypeWarning: Columns (7,9,13,14,15,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('blight_violations.csv')


## Apply Processing Function

In [3]:
df = pre_processing(df)

/Users/stanislas/Projet_Pierre/Projet_Pierre/ML_Deployment-Judgment_Outputs/my_model_package/forecasting/utils.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['violation_category'] = df['violation_description'].astype(str).apply(lambda x: map_violation_category(x))
/Users/stanislas/Projet_Pierre/Projet_Pierre/ML_Deployment-Judgment_Outputs/my_model_package/forecasting/utils.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['violation_category'] = df.apply(convert_to_other, axis=1)


In [4]:
data = df.copy()

## Print df

In [5]:
df.head()

,agency_name,violation_street_name,state,violation_date,hearing_date,hearing_time,judgment_date,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,balance_due,payment_status,violation_category,is_judgment_later
1,"Buildings, Safety Engineering & Env Department",Charlevoix,Mi,2006-11-08 00:00:00,2006-11-30 00:00:00,9:00AM,2006-11-30 10:23:59,Responsible by Default,250.0,20.0,10.0,0.0,0.0,280.0,280.0,NO PAYMENT,No Certificate,1
2,"Buildings, Safety Engineering & Env Department",Chrysler,Mi,2006-11-08 00:00:00,2006-11-30 00:00:00,9:00AM,2006-11-30 00:00:00,Responsible by Admission,250.0,20.0,10.0,0.0,0.0,280.0,280.0,PAID IN FULL,No Certificate,0
3,"Buildings, Safety Engineering & Env Department",St marys,Mi,2006-11-08 00:00:00,2006-12-13 00:00:00,3:00PM,2006-12-13 15:23:18,Responsible by Default,250.0,20.0,10.0,0.0,0.0,280.0,280.0,NO PAYMENT,No Certificate,1
6,"Buildings, Safety Engineering & Env Department",Warren,Mi,2006-11-08 00:00:00,2006-11-30 00:00:00,9:00AM,2006-11-30 10:23:03,Responsible by Default,250.0,20.0,10.0,0.0,0.0,280.0,280.0,NO PAYMENT,No Certificate,1
9,"Buildings, Safety Engineering & Env Department",Oakland,Mi,2006-11-08 00:00:00,2006-11-30 00:00:00,9:00AM,2006-11-30 10:26:26,Responsible by Default,250.0,20.0,10.0,0.0,0.0,280.0,280.0,NO PAYMENT,No Certificate,1


In [6]:
df.dropna(inplace=True)

## Feature Engineering

### Columns

In [7]:
df.columns

Index(['agency_name', 'violation_street_name', 'state', 'violation_date',
       'hearing_date', 'hearing_time', 'judgment_date', 'disposition',
       'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'judgment_amount', 'balance_due', 'payment_status',
       'violation_category', 'is_judgment_later'],
      dtype='object')

In [8]:
df.drop(columns=['disposition','violation_date','hearing_date','hearing_time','judgment_date'],inplace=True)

### Keep only the values with sufficent number of occurrences to avoid overfitting

#### Agency Name

In [9]:
df['agency_name'].value_counts()

agency_name
Buildings, Safety Engineering & Env Department    200552
BSEED Environmental Affairs                       153679
Detroit Police Department                          13778
Health Department                                   5466
Neighborhood City Halls                                1
Name: count, dtype: int64

With only 1 occurrence, Neighborhood City Halls must be removed to avoid training the model on such unique value.

In [10]:
df = df.query("agency_name != 'Neighborhood City Halls'")

#### Violation Street Name

In [11]:
count_street = df['violation_street_name'].value_counts()

In [12]:
count_street.describe()

count    1841.000000
mean      202.865291
std       454.125142
min         1.000000
25%        11.000000
50%        48.000000
75%       206.000000
max      6976.000000
Name: count, dtype: float64

Keep only the rows with occurrences>50 or remove this feature ?

#### State

In [13]:
state_count = df['state'].value_counts()

#### Fine Amount

In [14]:
df['fine_amount'].describe()

count    373475.000000
mean        310.436496
std         446.711317
min           1.000000
25%         100.000000
50%         250.000000
75%         250.000000
max       10000.000000
Name: fine_amount, dtype: float64

In [15]:
df = df.drop(columns=['fine_amount'])

#### Admin Fee

In [16]:
df['admin_fee'].describe()

count    373475.0
mean         20.0
std           0.0
min          20.0
25%          20.0
50%          20.0
75%          20.0
max          20.0
Name: admin_fee, dtype: float64

In [17]:
df['admin_fee'].value_counts()

admin_fee
20.0    373475
Name: count, dtype: int64

Admin Fee always has the same value.
It is better to remove this feature.

In [18]:
df = df.drop(columns=['admin_fee'])

#### State Fee

In [19]:
df['state_fee'].value_counts()

state_fee
10.0    373475
Name: count, dtype: int64

Same for State Fee, let's remove it as well.

In [20]:
df = df.drop(columns=['state_fee'])

#### Late Fee

In [21]:
df['late_fee'].value_counts()

late_fee
0.00      275678
25.00      32599
5.00       16699
10.00      13048
50.00      12959
           ...  
145.00         1
51.00          1
0.26           1
56.35          1
222.00         1
Name: count, Length: 126, dtype: int64

In [22]:
df['late_fee'].describe()

count    373475.000000
mean          8.040115
std          25.483829
min          -1.000000
25%           0.000000
50%           0.000000
75%           5.000000
max        1000.000000
Name: late_fee, dtype: float64

The values for late fees seeem to vary a lot. It's also better to remove it.

In [23]:
df = df.drop(columns=['late_fee'])

#### Discount Amount

In [24]:
df['discount_amount'].value_counts()

discount_amount
0.0       345907
25.0        8916
5.0         5948
10.0        5094
20.0        3466
50.0        2340
100.0        611
12.5         198
2.5          186
80.0         172
150.0        138
30.0         132
75.0          92
40.0          73
350.0         45
250.0         43
60.0          35
35.0          23
200.0         16
37.5          11
125.0         10
500.0          7
120.0          4
1000.0         3
255.0          2
930.0          1
300.0          1
3.0            1
Name: count, dtype: int64

It would be better to transform this information to identify the presence of a discount, and not its amount.

In [25]:
def amount_to_discount(row):
    if row['discount_amount']==0:
        return "No Discount"
    else:
        return "Discount"



In [26]:
df['discount_status'] = df.apply(amount_to_discount, axis=1)

In [27]:
df = df.drop(columns=['discount_amount'])

#### Judgment Amount

In [28]:
df['judgment_amount'].describe()

count    373475.000000
mean        355.372300
std         484.020992
min         -20.000000
25%         130.000000
50%         280.000000
75%         305.000000
max       23977.700000
Name: judgment_amount, dtype: float64

We should keep only the amounts superior to 0. 
Other amounts are not realistic or difficult to interpret.

In [29]:
df = df.query("judgment_amount>0")

In [30]:
df = df.drop(columns=['balance_due'])

## Final dataframe

In [31]:
df.columns

Index(['agency_name', 'violation_street_name', 'state', 'judgment_amount',
       'payment_status', 'violation_category', 'is_judgment_later',
       'discount_status'],
      dtype='object')

In [32]:
df.reset_index(inplace=True,drop=True)

In [33]:
df.head()

,agency_name,violation_street_name,state,judgment_amount,payment_status,violation_category,is_judgment_later,discount_status
0,"Buildings, Safety Engineering & Env Department",Charlevoix,Mi,280.0,NO PAYMENT,No Certificate,1,No Discount
1,"Buildings, Safety Engineering & Env Department",Chrysler,Mi,280.0,PAID IN FULL,No Certificate,0,No Discount
2,"Buildings, Safety Engineering & Env Department",St marys,Mi,280.0,NO PAYMENT,No Certificate,1,No Discount
3,"Buildings, Safety Engineering & Env Department",Warren,Mi,280.0,NO PAYMENT,No Certificate,1,No Discount
4,"Buildings, Safety Engineering & Env Department",Oakland,Mi,280.0,NO PAYMENT,No Certificate,1,No Discount


## Encode Categorical Values

In [34]:
to_encode = ['agency_name', 'violation_street_name', 'state', 'payment_status', 'violation_category','discount_status']

In [35]:
def label_encode(df):
    label_encoder = LabelEncoder()
    for col in to_encode:
        df[col] = label_encoder.fit_transform(df[col])
    return df

In [36]:
df = label_encode(df)

In [37]:
df.head()

,agency_name,violation_street_name,state,judgment_amount,payment_status,violation_category,is_judgment_later,discount_status
0,1,327,25,280.0,0,1,1,1
1,1,349,25,280.0,1,1,0,1
2,1,1591,25,280.0,0,1,1,1
3,1,1741,25,280.0,0,1,1,1
4,1,1281,25,280.0,0,1,1,1


## Export

In [38]:
df.to_csv('df_process.csv')